## Setting


In [ ]:
from src.model import Model
import torch
import torch.nn as nn

import os
import numpy as np
import tensorly as tl
from tensorly.decomposition import parafac, partial_tucker
from decomposition.decompositions import cp_decomposition_conv_layer, estimate_ranks, tucker_decomposition_conv_layer
from ptflops import get_model_complexity_info
from torchsummary import summary

In [ ]:
# switch to the PyTorch backend
tl.set_backend('pytorch')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
#mse error
def get_mse(src, tgt) -> float:
  """Calc mse."""
  if isinstance(src, torch.Tensor):
    return torch.mean((src - tgt)**2)
  elif isinstance(src, np.ndarray):
    return np.mean((src - tgt)**2)

#model complexity
def print_macs(layer: nn.Module,ch,row,col):
    macs, params = get_model_complexity_info(layer, tuple([ch,row,col]), as_strings=True,
                                           print_per_layer_stat=True, verbose=True)
    print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
    print('{:<30}  {:<8}'.format('Number of parameters: ', params))

## Model

In [ ]:
model_path = '/opt/ml/code/exp/latest/best.pt'
model_config = '/opt/ml/code/exp/latest/model.yml'

In [ ]:
model_instance = Model(model_config, verbose=True)
model_instance.model.load_state_dict(
    torch.load(model_path, map_location=torch.device("cpu"))
)
model = model_instance.model
#print_macs(model,3,512,512)
model = model.to('cpu')

In [ ]:
model

In [ ]:
print_macs(model,3,512,512)

## Tucker Decomposition

In [ ]:
#rank = [8,8]
rank = [0.2]
for i, module in enumerate(model.children()):
    if i == 0 or i == 16 or i == 18:
        module.conv = tucker_decomposition_conv_layer(module.conv,rank)
    elif 1<=i<=15:
        sub_layer = []
        
        for child_module in module[0].conv.children():
            if 'Squeeze' in str(type(child_module)):
                child_module.fc1 = tucker_decomposition_conv_layer(child_module.fc1,rank)
                child_module.fc2 = tucker_decomposition_conv_layer(child_module.fc2,rank)
                sub_layer.append(child_module)
            elif type(child_module) == nn.Conv2d:
                sub_layer.append(tucker_decomposition_conv_layer(child_module,rank))
            else:
                sub_layer.append(child_module)
        module[0].conv = nn.Sequential(*sub_layer)

In [ ]:
model

In [ ]:
print_macs(model,3,512,512)

In [ ]:
#save model
from src.utils.torch_utils import save_model
path = '/opt/ml/code/exp/latest_decom'
if not os.path.exists(path):
    os.mkdir(path)
save_model(model=model,path='/opt/ml/code/exp/latest_decom/best.pt',data=None,device=None)

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
im = np.array(Image.open('/opt/ml/data/test/NoLabel/00abmaytwoknyzdcatlabwpr.jpg'), dtype = np.uint8)
fig, ax = plt.subplots(1,1,figsize=(10,10))
ax.imshow(im)
im = torch.Tensor(im).permute(2,0,1).unsqueeze(0)
print(im.shape)
im = im.to(device)

In [ ]:
m = nn.Sequential(
    nn.Conv2d(3,10,3),
    nn.BatchNorm1d(10),
    nn.ReLU(),
    nn.Conv2d(10,3,3),
    nn.ReLU()
)

In [ ]:
layers = []

In [ ]:
for key in m:
    if type(key) == nn.Conv2d:
        layers.append(tucker_decomposition_conv_layer(key))
    else:
        layers.append(key)

In [ ]:
m.add_module('new',nn.ReLU())

In [ ]:
m.state_dict

In [ ]:
for key in m:
    print(key)
    if type(key) == nn.Conv2d:
        key = nn.ReLU()

for key in m:
    print(key)

In [ ]:
for i in dd:
    print(i)
    #print(d[i])
    #break

In [ ]:
for key in m:
    print(key.weight)
    break

In [ ]:
for key in m:
    print(key)
    if type(key) == nn.Conv2d:
        print(key.weight)
        key.weight[0][0][0][0] = 0
        
